In [1]:
import re, sys, os
import logging
from pathlib import Path
from datetime import datetime
import numpy as np
from plum import dispatch
import torch
import torch_directml
from transformers import *
from fastai.text.all import *
from fastai.data.all import *
import pandas as pd

torch._logging.set_logs(all=logging.WARNING)

🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py


In [2]:
# Setup DirectML
print("="*80)
print("DIRECTML FULL DEBUG LOG STARTED")
print(f"torch version: {torch_directml.torch.__version__}")
print(f"Device: {torch_directml.device()}")
print(f"GPU: {torch_directml.device_name(0)}")
print("="*80)

dml = torch_directml.device()

# Global monkey-patch for Normalize
old_init = Normalize.__init__
def new_init(self, mean, std, axes=(0,2,3)):
    old_init(self, mean.to(dml), std.to(dml), axes=axes)
Normalize.__init__ = new_init

DIRECTML FULL DEBUG LOG STARTED
torch version: 2.4.1+cpu
Device: privateuseone:0
GPU: Radeon RX 560X 


In [3]:
# =====================================================
# 1. Monkey-patch TensorText to add truncate and show methods
# =====================================================
@patch
def truncate(self: TensorText, n):
    """Truncate TensorText to n tokens"""
    return type(self)(self[:n])

@patch  
def show(self: TensorText, ctx=None, **kwargs):
    """Show decoded text"""
    # Get the tokenizer from kwargs or use a stored one
    tokenizer = kwargs.get('tokenizer', getattr(self, '_tokenizer', None))
    
    if tokenizer is not None:
        # Decode using HF tokenizer
        text = tokenizer.decode(self.cpu().tolist(), skip_special_tokens=True)
    else:
        # Fallback: just show token IDs
        text = str(self.cpu().tolist())
    
    if ctx is None:
        print(text)
        return text
    return show_title(text, ctx=ctx, **kwargs)

# =====================================================
# 2. Simple Transform - just tokenize to input_ids
# =====================================================
class HFTokenize(Transform):
    """Transform that tokenizes text files using HuggingFace tokenizer"""
    def __init__(self, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length
        # Store tokenizer globally for TensorText.show() to access
        TensorText._hf_tokenizer = tokenizer
    
    def encodes(self, path: Path):
        """Read file and tokenize, returning TensorText with just input_ids"""
        text = path.read_text(encoding='utf-8') if path.exists() else ""
        
        # Tokenize - get tensors directly from tokenizer
        encoding = self.tokenizer(
            text,
            truncation='longest_first',
            max_length=self.max_length,
            padding='max_length',
            add_special_tokens=True,
            return_tensors='pt'  # Return PyTorch tensors
        )
        
        # Squeeze batch dimension [1, seq_len] -> [seq_len]
        tensor_text = TensorText(encoding['input_ids'].squeeze(0))
        # Attach tokenizer for later decoding
        tensor_text._tokenizer = self.tokenizer
        return tensor_text
    
    def decodes(self, o):
        """Keep as TensorText for show_batch compatibility"""
        # Don't decode to string - keep as TensorText
        # The actual text decoding happens in show() method
        return o

# =====================================================
# 2. Callback that computes attention_mask on-the-fly
# =====================================================
class HFCallback(Callback):
    """
    Computes attention_mask from padded input_ids and injects into model.
    This is the ONLY place where HuggingFace-specific logic lives.
    """
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
    
    def before_batch(self):
        """Compute attention_mask from input_ids before each forward pass"""
        # Get input_ids from batch
        xb = self.xb[0] if isinstance(self.xb, tuple) and len(self.xb) > 0 else self.xb
        
        # Compute attention_mask: 1 for real tokens, 0 for padding
        attention_mask = (xb != self.pad_idx).long()
        
        # Inject into model for this forward pass
        self.model._attention_mask = attention_mask

# =====================================================
# 3. Model wrapper that uses injected attention_mask
# =====================================================
class HFModelWrapper(Module):
    """Wrapper to adapt HF model to FastAI"""
    def __init__(self, hf_model, pad_idx):
        self.hf_model = hf_model
        self.pad_idx = pad_idx
        self._attention_mask = None
    
    def forward(self, input_ids):
        # Get attention_mask from callback (or compute default)
        attention_mask = getattr(self, '_attention_mask', None)
        if attention_mask is None:
            attention_mask = (input_ids != self.pad_idx).long()
        
        # Clear after use (for next batch)
        self._attention_mask = None
        
        # Forward pass with attention mask
        outputs = self.hf_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        return outputs.logits

In [4]:
# =====================================================
# 5. Setup DataLoaders and Model
# =====================================================
os.makedirs("models", exist_ok=True)
model_cache_dir = Path("models")
hf_model = "haisongzhang/roberta-tiny-cased"

# Load HF model and tokenizer
hf_model_obj = AutoModelForSequenceClassification.from_pretrained(
    hf_model, num_labels=2, cache_dir=model_cache_dir,
    use_safetensors=True
)
tokenizer = AutoTokenizer.from_pretrained(hf_model)
""

loading configuration file config.json from cache at models\models--haisongzhang--roberta-tiny-cased\snapshots\2e8caf4404987b8cb20fa4b22955f56940b2ebc6\config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.57.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

Attempting to create safetensors variant
Safetensors PR exists
loading weights file model.safetensors from cache at models\models--haisongzhang--roberta-tiny-cased\snapshots\bfd314bd7663eef65c83e57719c0d9f3c21aa631\model.sa

''

In [5]:
# Load data
path = untar_data(URLs.IMDB)

# Create DataBlock with HFTransform
blocks = (
    TransformBlock(
        type_tfms=HFTokenize(tokenizer, max_length=512),
        dls_kwargs={'before_batch': Pad_Chunk(seq_len=512, pad_idx=tokenizer.pad_token_id)}
    ),
    CategoryBlock
)

dblock = DataBlock(
    blocks=blocks,
    get_items=get_text_files,
    get_y=parent_label,
    splitter=GrandparentSplitter(valid_name='test')
)

In [6]:
# Create dataloaders
dls = dblock.dataloaders(path, bs=1, device=dml, verbose=True, num_workers=0)

Found 100002 items
2 datasets of sizes 25000,25000
Setting up Pipeline: HFTokenize
Setting up Pipeline: parent_label -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Setting up after_item: Pipeline: ToTensor
Setting up before_batch: Pipeline: Pad_Chunk -- {'pad_idx': 0, 'pad_first': True, 'seq_len': 512}

Setting up after_batch: Pipeline: 


In [7]:
dls.show_batch(max_n=2)

,None,category
0,"cross - eyed is a very original and funny movie. i think adam jones brings a refreshing new set of eyes to the comedy genre and really reinvents it in a good way. this film is smart, concise, and consistently entertaining and funny. as a writer / director, jones exhibits complete control over his characters who are both absurd and lovable. the story is definitely something you haven't seen before which is good. it's unique and fun, and manages to work in visually fantastic elements as well as the long lost slapstick genre together to form a hearty comedy. < br / > < br / > a very promising first film.",pos


In [8]:
# =====================================================
# 7. Create Learner with custom callback
# =====================================================
# Wrap the HF model
model_wrapped = HFModelWrapper(hf_model_obj, tokenizer.pad_token_id)
model_wrapped.to(dml)

# Create learner
learn = Learner(
    dls, 
    model_wrapped,
    loss_func=CrossEntropyLossFlat(),
    metrics=accuracy,
    cbs=[HFCallback(pad_idx=tokenizer.pad_token_id)]
).to_fp16(enabled=False)

learn.to(dml)
print("\n" + "="*80)
print("LEARNER CREATED SUCCESSFULLY")
print("="*80)


LEARNER CREATED SUCCESSFULLY


In [9]:
print("\n" + "="*80)
print("VERIFYING BATCH STRUCTURE")
print("="*80)

try:
    batch = dls.one_batch()
    xb, yb = batch
    
    print(f"Batch shape: {xb.shape}")
    print(f"Batch dtype: {xb.dtype}")
    print(f"First item type: {type(xb[0])}")
    print(f"Labels shape: {yb.shape}")
    print(f"Pad token ID: {tokenizer.pad_token_id}")
    print(f"Sample tokens (first 20): {xb[0][:20]}")
    
    # Show padding
    pad_count = (xb[0] == tokenizer.pad_token_id).sum().item()
    print(f"Padding tokens in first sample: {pad_count}/{len(xb[0])}")
    
    print("\n✓ Batch structure correct!")
except Exception as e:
    print(f"\n✗ Batch verification error: {e}")
    import traceback
    traceback.print_exc()


VERIFYING BATCH STRUCTURE
Batch shape: torch.Size([1, 512])
Batch dtype: torch.int64
First item type: <class 'fastai.text.data.TensorText'>
Labels shape: torch.Size([1])
Pad token ID: 0
Sample tokens (first 20): TensorText([  101,  1103,  1825,  1150,  1724,  1103, 10140,  3189,  1104,
             1142,  1940,  1116, 13830, 10517,  1933,  1538,  1129,  2272,
             1106,  1103], device='privateuseone:0')
Padding tokens in first sample: 352/512

✓ Batch structure correct!


In [11]:
import cProfile
import pstats
from io import StringIO

# Profile the training
profiler = cProfile.Profile()
profiler.enable()
learn.fine_tune(1, base_lr=4e-3)
profiler.disable()

f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,time
0,0.691856,0.693141,0.500000,4:39:54


epoch,train_loss,valid_loss,accuracy,time
0,0.695306,0.693142,0.500000,4:37:21


In [12]:
# Get top bottlenecks
s = StringIO()
ps = pstats.Stats(profiler, stream=s).sort_stats('cumulative')
ps.print_stats(30)  # Top 30 functions
print(s.getvalue())

         1704328515 function calls (1564952522 primitive calls) in 33434.991 seconds

   Ordered by: cumulative time
   List reduced from 1063 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   506466    7.704    0.000 123800.188    0.244 C:\Users\Keshav\.pyenv\pyenv-win\versions\3.12.9\Lib\asyncio\base_events.py:1922(_run_once)
   506741    1.888    0.000 48105.548    0.095 C:\Users\Keshav\.pyenv\pyenv-win\versions\3.12.9\Lib\asyncio\events.py:86(_run)
      3/2    0.000    0.000 33436.142 16718.071 f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\fastai\learner.py:258(_do_fit)
   100004    3.590    0.000 32926.898    0.329 f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\fastai\data\load.py:125(__iter__)
 12707876  117.812    0.000 25874.254    0.002 f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\fastai\torch_core.py:380(__torch_function__)
 12707876  121.500 

In [17]:
test_path = Path("test_reviews/test_1.txt")
learn.predict(test_path)

f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
f:\Projects\VsCodium\Python\2025\GenAI\11\.venv3_12_9\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


('pos', TensorText(1), TensorText([0.4962, 0.5038]))